In [2]:
import pickle
import glob
import os
import numpy as np
import pandas as pd
import glob
from nilearn import plotting
from atlasreader import create_output
from pathlib import Path

import matplotlib 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import rcParams
%matplotlib inline

The Python package you are importing, AtlasReader, is licensed under the
BSD-3 license; however, the atlases it uses are separately licensed under more
restrictive frameworks.
By using AtlasReader, you agree to abide by the license terms of the
individual atlases. Information on these terms can be found online at:
https://github.com/miykael/atlasreader/tree/master/atlasreader/data



In [3]:
path_to_first_level = Path("/work/LauraBockPaulsen#1941/fMRI_analysis/flms")
flm_files = [f for f in path_to_first_level.iterdir() if f.is_file()]

flms = []

for model in flm_files: # looping over the flms and loading them
    flm_model = pickle.load(open(model,'rb')) 
    flms.append(flm_model)